In [ ]:
# run(`conda create --channel conda-forge --channel bioconda --channel defaults --strict-channel-priority --name blast blast`)

In [ ]:
import Pkg
pkgs = [
    "Revise",
    "DataFrames",
    "uCSV",
    "StatsPlots"
    "ProgressMeter",
    "StatsBase",
    "FASTX"
]
# Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end
import Mycelia

In [ ]:
data_dir = joinpath(dirname(pwd()), "data")

In [ ]:
# total_memory = 64
# threads = max(1, Int(total_memory//8))

total_memory = 128
threads = max(1, Int(total_memory//16))

memory = Int(total_memory//threads)

In [ ]:
blast_task = "megablast"

In [ ]:
db = "nt"

In [ ]:
blastdbs_dir = mkpath(joinpath(homedir(), "workspace", "blastdbs"))

In [ ]:
blast_db = joinpath(blastdbs_dir, db)

In [ ]:
Mycelia.download_blast_db(db="nt", outdir=blastdbs_dir, source="ncbi", conda_env="blast")

In [ ]:
# load in metadata
metadata_dir = joinpath(dirname(pwd()), "metadata")

exposome_environmental_data = DataFrames.DataFrame(uCSV.read(
    joinpath(metadata_dir, "metadata_exposome.rds.tsv"),
    delim='\t',
    header=1,
    typedetectrows=300
))

joint_sample_metadata = DataFrames.DataFrame(uCSV.read(
    joinpath(metadata_dir, "exposome/joint_sample_metadata.tsv"),
    delim='\t',
    header=1,
    typedetectrows=300
))

@assert joint_sample_metadata[!, "Library Name"] == joint_sample_metadata[!, "LibraryName"]

joint_metadata = DataFrames.innerjoin(
    joint_sample_metadata,
    exposome_environmental_data,
    on="Library Name" => "samplenames")

run_ids = sort(joint_metadata[!, "Run"])

sample_paths = joinpath.(data_dir, "SRA", run_ids)

In [ ]:
# sample_paths = filter(x -> !occursin(".ipynb_checkpoints", x), readdir(joinpath(data_dir, "SRA"), join=true))
# sample_paths = filter(x -> isfile(joinpath.(x, "megahit", "final.contigs.fastg.gfa.fna")), sample_paths)

In [ ]:
# find the small outliers on output size and run again

In [ ]:
file_sizes = []
for sample_path in sample_paths[1:end]
    sample = basename(sample_path)
    blastn_directory = mkpath(joinpath(sample_path, "blastn"))
    assembled_fasta = joinpath(sample_path, "megahit", "final.contigs.fastg.gfa.fna")
    target_outfile = joinpath(blastn_directory, basename(assembled_fasta) * ".blastn.$(db).$(blast_task).txt")
    # --mem=$(memory)G
    if isfile(target_outfile)
            # @show sample, filesize(target_outfile)
        push!(file_sizes, filesize(target_outfile))
    else
        @show sample
    end

    # cmd = "julia $(homedir())/workspace/Mycelia/scripts/run-blastn.jl --fasta $(assembled_fasta) --blast_db $(blast_db) --task $(blast_task) --out_directory $(blastn_directory) --threads $(threads)"
    # submission = 
    # `sbatch 
    # --mail-user=cameron.prybol@gmail.com
    # --mail-type=ALL
    # --error=%x-%j.err
    # --output=%x-%j.out
    # --partition=batch
    # --account=mpsnyder
    # --nodes=1
    # --ntasks=1
    # --time=1-00:00:00    
    # --cpus-per-task=$(threads)
    # --mem-per-cpu=$(memory)G
    # --job-name=blastn
    # --wrap $cmd
    # `
    # run(submission)
    # end
end
# count(file_sizes .== 0)

In [ ]:
StatsPlots.histogram(file_sizes, nbins=80)

In [ ]:
# findlast(sample_path -> !isempty(readdir(joinpath(sample_path, "blastn"))), sample_paths)

In [ ]:
# sample_paths = filter(x -> !occursin(".ipynb_checkpoints", x), readdir(joinpath(data_dir, "SRA"), join=true))
# sample_paths = filter(x -> isfile(joinpath(x, "megahit", "final.contigs.fastg.gfa.fna")), sample_paths)

# for sample_path in sample_paths
#     sample = basename(sample_path)
#     blastn_directory = joinpath(sample_path, "blastn")
#     @show readdir(blastn_directory)
#     target_outfile = joinpath(sample_path, "blastn", "final.contigs.fastg.gfa.fna.blastn.nt.$(blast_task).txt")
#     # || filesize(target_outfile) == 0
#     if !isfile(target_outfile) 
#         assembled_fasta = joinpath(sample_path, "megahit", "final.contigs.fastg.gfa.fna")
#         cmd = "julia $(homedir())/workspace/Mycelia/scripts/run-blastn.jl --fasta $(assembled_fasta) --blast_db nt --task $(blast_task) --out_directory $(blastn_directory) --threads $(threads)"
#         submission = 
#         `sbatch 
#         --mail-user=cameron.prybol@gmail.com
#         --mail-type=ALL
#         --error=%x-%j.err
#         --output=%x-%j.out
#         --partition=batch
#         --account=mpsnyder
#         --nodes=1
#         --ntasks=1
#         --time=1-00:00:00    
#         --cpus-per-task=$(threads)
#         --mem=$(memory)G
#         --job-name=blastn
#         --wrap $cmd
#         `
#         run(submission)
#         # end
#     end
# end

In [ ]:
# function parse_blast_report(blast_report)
#     # example header line 
#     # "# Fields: query id, subject id, subject acc., subject acc.ver, subject title, query length, subject length, q. start, q. end, s. start, s. end, evalue, bit score, score, alignment length, % identity, identical, mismatches, subject tax id"
#     header_lines = collect(Iterators.filter(x -> occursin(r"# Fields:", x), eachline(blast_report)))
#     if isempty(header_lines)
#         @info "no hits found, returning empty table"
#         return DataFrames.DataFrame()
#     end
#     header_line = first(header_lines)
#     header = split(last(split(header_line, ": ")), ", ")
#     blast_col_types = Dict(
#         "query id" => String,
#         "query title" => String,
#         "subject id" => String,
#         "subject gi" => String,
#         "subject acc." => String,
#         "subject acc.ver" => String,
#         "subject title" => String,
#         "query length" => Int,
#         "subject length" => Int,
#         "q. start" => Int,
#         "q. end" => Int,
#         "s. start" => Int,
#         "s. end" => Int,
#         "evalue" => Float64,
#         "bit score" => Float64,
#         "score" => Float64,
#         "alignment length" => Int,
#         "% identity" => Float64,
#         "identical" => Int,
#         "mismatches" => Int,
#         "subject tax id" => Int,
#         "subject sci name" => String,
#         "subject com names" => String,
#         "subject blast name" => String,
#         "subject super kingdom" => String,
#         "subject tax ids" => String,
#         "subject sci names" => String,
#         "subject com names" => String,
#         "subject blast names" => String,
#         "subject super kingdoms" => String,
#         "subject title" => String,
#         "subject titles" => String
#     )
#     data, _ = uCSV.read(
#         blast_report,
#         delim='\t',
#         comment='#',
#         # skipmalformed=true,
#         allowmissing=true,
#         encodings=Dict("N/A" => missing),
#         types=[blast_col_types[h] for h in header])
#     return DataFrames.DataFrame(data, header, makeunique=true)
# end

In [ ]:
viral_tax_ids = Set(parse.(Int, filter(!isempty, readlines(`conda run -n taxonkit taxonkit list --ids 10239 --indent ""`))))

In [ ]:
basedir = "/labs/mpsnyder/share/exposome_data"

In [ ]:
megablast_results = String[]
blastn_results = String[]
for batch_directory in [
    joinpath(basedir, "extracted1"),
    joinpath(basedir, "Expo_pliot1_extracted_hifi_fastqs")
]
    for x in filter(x -> occursin(r"\.nt\.megablast\.txt$", x), readdir(batch_directory, join=true))
        push!(megablast_results, x)
    end
    for x in filter(x -> occursin(r"\.nt\.blastn\.txt$", x), readdir(batch_directory, join=true))
        push!(blastn_results, x)
    end
end

In [ ]:
joint_blast_results_table = DataFrames.DataFrame()

In [ ]:
    blast_col_types = Dict(
        "query id" => String,
        "query title" => String,
        "subject id" => String,
        "subject gi" => String,
        "subject acc." => String,
        "subject acc.ver" => String,
        "subject title" => String,
        "query length" => Int,
        "subject length" => Int,
        "q. start" => Int,
        "q. end" => Int,
        "s. start" => Int,
        "s. end" => Int,
        "evalue" => Float64,
        "bit score" => Float64,
        "score" => Float64,
        "alignment length" => Int,
        "% identity" => Float64,
        "identical" => Int,
        "mismatches" => Int,
        "subject tax id" => Int,
        "subject sci name" => String,
        "subject com names" => String,
        "subject blast name" => String,
        "subject super kingdom" => String,
        "subject tax ids" => String,
        "subject sci names" => String,
        "subject com names" => String,
        "subject blast names" => String,
        "subject super kingdoms" => String,
        "subject title" => String,
        "subject titles" => String
    )

In [ ]:
ProgressMeter.@showprogress for blast_report in megablast_results
    header_line = first(Iterators.filter(x -> occursin(r"# Fields:", x), eachline(blast_report)))
    header = split(last(split(header_line, ": ")), ", ")
    viral_lines = String[]
    for line in eachline(blast_report)
        if !occursin("#", line)
            subject_tax_ids = map(x -> parse(Int, x), split(last(split(line, '\t')), ';'))
            if any(x -> x in viral_tax_ids, subject_tax_ids)
                push!(viral_lines, line)
            end
        end
    end
    data, _ = uCSV.read(IOBuffer(join(viral_lines, '\n')), delim='\t')
    results_table = DataFrames.DataFrame(data, header)
    results_table[!, "method"] .= "megablast"
    results_table[!, "sample"] .= replace(basename(blast_report), ".bam.fasta.blastn.nt.megablast.txt" => "")
    append!(joint_blast_results_table, results_table, promote=true)
end

In [ ]:
ProgressMeter.@showprogress for blast_report in blastn_results
    try
        header_line = first(Iterators.filter(x -> occursin(r"# Fields:", x), eachline(blast_report)))
        header = split(last(split(header_line, ": ")), ", ")
        viral_lines = String[]
        for line in eachline(blast_report)
            if !occursin("#", line)
                subject_tax_ids = map(x -> parse(Int, x), split(last(split(line, '\t')), ';'))
                if any(x -> x in viral_tax_ids, subject_tax_ids)
                    push!(viral_lines, line)
                end
            end
        end
        data, _ = uCSV.read(IOBuffer(join(viral_lines, '\n')), delim='\t', types=[blast_col_types[h] for h in header])
        results_table = DataFrames.DataFrame(data, header)
        results_table[!, "method"] .= "blastn"
        results_table[!, "sample"] .= replace(basename(blast_report), ".bam.fasta.blastn.nt.blastn.txt" => "")
        append!(joint_blast_results_table, results_table, promote=true)
    catch
        println("skipping $(blast_report)")
    end
end

In [ ]:
StatsBase.describe(joint_blast_results_table[!, "alignment length"])

In [ ]:
# StatsPlots.histogram(joint_blast_results_table[!, "alignment length"])

In [ ]:
sort!(joint_blast_results_table, "alignment length", rev=true)

In [ ]:
joint_blast_results_table_filtered = joint_blast_results_table[joint_blast_results_table[!, "alignment length"] .> 1000, :]

In [ ]:
sequence_identifiers_of_interest = Set(unique(joint_blast_results_table_filtered[!, "query id"]))

In [ ]:
fasta_sequences = []
ProgressMeter.@showprogress for fasta_file in replace.(megablast_results, ".blastn.nt.megablast.txt" => "")
    for record in FASTX.FASTA.Reader(open(fasta_file))
        if FASTX.identifier(record) in sequence_identifiers_of_interest
            push!(fasta_sequences, record)
        end
    end
end
fasta_sequences

In [ ]:
# i = 1 labrador retriever
# i = 2 unknown
# i = 3 bacteriophage
# i = 4 bacteriophage
# i = 5 dog
# i = 6 dog
# i = 7 dog
# i = 8 dog
# i = 9 dog
# i = 10 dog
# i = 11 e. coli or e. coli phage
# i = 12 e. coli or e. coli phage
# i = 13 e. coli or e. coli phage
# i = 14  e. coli or e. coli phage
# i = 15  e. coli or e. coli phage
# i = 16  e. coli or e. coli phage
record = fasta_sequences[i]
println(record)